In [ ]:
import numpy as np
import pickle
import os
import glob
import re
import sys
from tabulate import tabulate
from pathlib import Path
print('test')
from spectral_cube import SpectralCube

from photutils.aperture import CircularAperture, aperture_photometry
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from scipy.ndimage import median_filter

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.patches import Circle, Rectangle
from matplotlib.gridspec import GridSpec
plt.rcParams['font.family'] = 'DejaVu Sans'

from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from astropy.wcs import WCS
from astropy.constants import c
from astropy.io import fits
from astropy.visualization import simple_norm, imshow_norm
from astropy.visualization import AsinhStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import SqrtStretch 

#TJ define functions needed to generate files
os.chdir('/d/ret1/Taylor/jupyter_notebooks/Research') #TJ change working directory to be the parent directory
from Py_files.Basic_analysis import *
from Py_files.All_flux_calibration_functions import *
filter_file = '/d/crow1/tools/cigale/database_builder/filters/jwst/nircam/F405N.dat'

In [ ]:
filter_data = []
with open(filter_file, 'r') as f:
    header = f.readline().strip().split()
    for line in f:
        data_line = line.strip().split()
        filter_data.append(data_line)
if len(filter_data) < 2:
    print(f"Filter file {filter_file} seems empty or malformed.")

header, filter_T = filter_data[:2], np.array(filter_data[2:])
filter_wl = np.array([try_float(row[0]) * 1e-10 for row in filter_T])
filter_trans = np.array([try_float(row[1]) for row in filter_T])

files = glob.glob('Data_files/IFU_files/F405N_contamination/*f290l*')
_, wl2 = get_filter_wl_range('F405N')
wl1 = 3.96816e-6 * u.m
for file in files:
    print(file)
    cube = SpectralCube.read(file, hdu = 'SCI').with_spectral_unit(u.m)
    subcube = cube.spectral_slab(wl1, wl2)
    header = fits.open(file)['SCI'].header
    print(subcube.shape)
files[0]
header

In [ ]:
#This method did not work

'''show_all_plots = False
show_voigt_fit = False
show_continuum_approx = False
show_continuum_wo_br_a = True
show_normalized_flux = False

# Procedure. Loop through each pixel doing the follwing:
# Find the median continuum flux level and the noise in the cube if we ignore the region around the Br-a region
# If the continuum flux is less than 3 times the noise in that pixel, skip this pixel
# Fit a voigt to the Br-a feature (this is designed to subtract the continuum and THEN fit, so the voigt goes to zero outside the feature
# Subtract the voigt line from the entire flux array, this is now the flux if we ignore the Br-a feature
# Subtract the continuum from the flux array without Br-a, this should be about zero, but with contaminate fluxes still present
# Subtract the continuum from the flux array that still contains Br-a.
# Apply the filter transmission and integrate both of the above arrays. One is the continuum subtracted flux (including Br-a), one is just contaminates
#
for file in files:
#TJ initialize output arrays that will be saved at the end of this cell
    obs_flux = []
    fluxes_wo_bra = []
    contamination = []
    discard_mask = []
    bra_flux = []
    #TJ cut cube so only filter's wavelengths are included
    cube = SpectralCube.read(file, hdu = 'SCI').with_spectral_unit(u.m)
    subcube = cube.spectral_slab(wl1, wl2)
    #TJ set minimum continuum flux over continuum noise ratio to include pixel in analysis
    sigma_threshold = 3
    #TJ loop through all pixels
    for x_pix in range(0,subcube.shape[2]):
        for y_pix in range(0,subcube.shape[1]):
            #TJ assign flux array for this pixel
            flux = subcube[:, y_pix, x_pix]
            flux = np.array(flux.value, dtype=float)
            #TJ ignore pixels that include nans anywhere
            if sum(np.isnan(flux)) > 1:
                continue
            wl = subcube.spectral_axis.value
            Br_mask = np.ones_like(flux, dtype=bool)
            #TJ create mask to ignore region directly around the Br-a feature so noise isnt super high whenever this feature is strong
            Br_mask[(wl>40500e-10) & (wl<40700e-10)] = False
            #TJ assign running continuum and noise in flux array (outside the Br-a feature)
            continuum = median_filter(flux, size=30, mode='reflect')
            raw_noise = np.std(flux[Br_mask])
            if np.median(flux[Br_mask])/raw_noise > sigma_threshold:
                color = 'blue'
            else:
                color = 'red'
                continue
            #TJ create normalized flux array (not generally used, but can be useful)
            Nflux = flux/continuum
        
            #TJ find peaks outside of 
            peaks, _ = find_peaks(Nflux, height=1+(sigma_threshold * np.std(Nflux[Br_mask])))
            # Step 4. Mask out preliminary peaks to estimate continuum noise robustly
            mask = np.ones_like(flux, dtype=bool)
            mask[peaks] = False
            noise = np.std(Nflux[Br_mask&mask])
            x = np.linspace(wl[0], wl[-1], 1000)
            flux_HD = np.interp(x, wl, flux)
            br_a = 0
            for peak in peaks:
                if np.isclose(wl[peak], 40580e-10, 0.001):
                    Nbr_a = fit_voigt_to(wl[peak], 1, wl, Nflux, show_plot = (show_all_plots or show_voigt_fit), feature_idx_width=10)
                    br_a = fit_voigt_to(wl[peak], 1, wl, flux, show_plot = (show_all_plots or show_voigt_fit), feature_idx_width=10)
                    br_a_x, br_a_y = br_a[0]
                    line = voigt(x, *br_a[4])
    
                    flux_wo_br_a = flux_HD - line
                    br_a_continuum = np.interp(br_a_x, wl, continuum)
                    continuum_HD = np.interp(x, wl, continuum)
                    continuum_wo_br_a = median_filter(flux_wo_br_a, size=300, mode='reflect')
                    residuals = flux_HD - continuum_HD - line
                    if np.median(flux[Br_mask])/raw_noise > sigma_threshold:
                        discard_mask.append(True)
                    else:
                        discard_mask.append(False)
                    bra_flux.append(br_a[1])
                    obs_flux.append(get_Fnu_transmission(flux_HD, x, filter_trans, filter_wl))
                    fluxes_wo_bra.append(get_Fnu_transmission(flux_wo_br_a, x, filter_trans, filter_wl))
                    contamination.append(get_Fnu_transmission((flux_wo_br_a - continuum_wo_br_a), x, filter_trans, filter_wl))
                    print()
            if br_a == 0:
                print(f'bracket alpha not found in pixel {x_pix,y_pix}')
                continue
            flux_no_br = flux
            if show_all_plots or show_normalized_flux:
                plt.plot(wl, Nflux, color = color)
                plt.axhline(y = 1, alpha = 0.3, linewidth = 2, linestyle = '--', color = 'black')
                plt.axvline(x = 40587e-10, alpha = 0.3, linewidth = 2, linestyle = '--', color = 'black', label = 'Br-a')
                plt.xlabel('wavelength (angstroms)')
                plt.ylabel('Normalized FLux (MJy/sr)')
                plt.title(f'Normalized spectrum for pixel x,y = {x_pix,y_pix}, f/n = {np.median(flux[Br_mask]):.2f}/{raw_noise:.2f}')
                plt.ylim(0,2)
                plt.legend()
                plt.show()
            if show_all_plots or show_voigt_fit:
                plt.plot(wl, flux, color = color)
                plt.plot(br_a_x, br_a_y+br_a_continuum, color = 'green', label = 'fitted voigt')
                plt.axvline(x = 40587e-10, alpha = 0.3, linewidth = 2, linestyle = '--', color = 'black', label = 'Br-a')
                plt.xlabel('wavelength (angstroms)')
                plt.ylabel('Normalized FLux (MJy/sr)')
                plt.title(f'Spectrum for pixel x,y = {x_pix,y_pix}, f/n = {np.median(flux[Br_mask]):.2f}/{raw_noise:.2f}')
                plt.show()
            if show_all_plots or show_continuum_wo_br_a:
                plt.plot(x, continuum_wo_br_a, color = 'red')
                plt.plot(x, flux_wo_br_a, color = 'green', label = 'flux')
                plt.plot(x, flux_HD, color = 'black', label = 'before feature subtraction')
                plt.plot(br_a_x, br_a_y+br_a_continuum, color = color, label = 'voigt')
                plt.axvline(x = 40587e-10, alpha = 0.3, linewidth = 2, linestyle = '--', color = 'black', label = 'Br-a')
                plt.xlabel('wavelength (angstroms)')
                plt.ylabel(f'FLux (MJy/sr)\npeak at {max(flux_HD)}')
                plt.title(f'Spectrum for pixel x,y = {x_pix,y_pix}, f/n = {np.median(flux[Br_mask]):.2f}/{raw_noise:.2f}')
                plt.ylim(min(flux_wo_br_a), max(flux_wo_br_a))
                plt.legend()
                plt.show()
    print(f'Loop stopped with pixel {x_pix}, {y_pix}')
    obs_flux = np.array(obs_flux)
    contamination = np.array(contamination)
    fluxes_wo_bra = np.array(fluxes_wo_bra)
    bra_flux = np.array(bra_flux)
    np.save(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_br_a_flux.npy', bra_flux)
    np.save(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_observed.npy', obs_flux)
    np.save(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_contamination.npy', contamination)
    np.save(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_flux_wo_br_a.npy', fluxes_wo_bra)'''
print()

In [ ]:
#plot the results
for file in files:
    prefix = file.split('jw03435-')[-1][:9]
    br_a_flux = np.load(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_br_a_flux.npy')
    obs_flux = np.load(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_observed.npy')
    contamination = np.load(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_contamination.npy')
    plt.scatter(obs_flux, contamination/obs_flux, marker = 'x', s = 1)
    median = np.nanmedian(contamination/obs_flux)
    upper = np.nanpercentile(contamination/obs_flux, 68)
    lower = np.nanpercentile(contamination/obs_flux, 33)
    plt.axhline(y = median)
    plt.axhline(y = upper, linestyle = '--')
    plt.axhline(y = lower, linestyle = '--')
    plt.title(f'Contamination analysis of {prefix}')
    plt.xlabel('Total Observed Flux (MJy/sr)')
    plt.ylabel('Ratio of Br-a flux to contaminates')
    plt.ylim(lower*0.5, upper*2)
    plt.show()


In [ ]:
wl[right_idx-1]-wl[left_idx+1]


In [ ]:
#testing more barebones integration tactics
show_all_plots = False
show_voigt_fit = False
show_co_fluxntinuum_approx = False
show_continuum_wo_br_a = False
show_normalized_flux = False

for file in files:
#TJ initialize output arrays that will be saved at the end of this cell
    obs_flux = []
    fluxes_wo_bra = []
    contamination = []
    discard_mask = []
    bra_flux = []
    #TJ cut cube so only filter's wavelengths are included
    cube = SpectralCube.read(file, hdu = 'SCI').with_spectral_unit(u.m)
    subcube = cube.spectral_slab(wl1, wl2)
    #TJ set minimum continuum flux over continuum noise ratio to include pixel in analysis
    sigma_threshold = 3
    #TJ loop through all pixels
    for x_pix in range(0,subcube.shape[2]):
        for y_pix in range(0,subcube.shape[1]):
            #TJ assign flux array for this pixel
            flux = subcube[:, y_pix, x_pix]
            flux = np.array(flux.value, dtype=float)
            #TJ ignore pixels that include nans anywhere
            if sum(np.isnan(flux)) > 1:
                continue
            wl = subcube.spectral_axis.value
            Br_mask = np.ones_like(flux, dtype=bool)
            #TJ create mask to ignore region directly around the Br-a feature so noise isnt super high whenever this feature is strong
            Br_mask[(wl>40500e-10) & (wl<40700e-10)] = False
            #TJ assign running continuum and noise in flux array (outside the Br-a feature)
            continuum = median_filter(flux, size=30, mode='reflect')
            raw_noise = np.std(flux[Br_mask])
            if np.median(flux[Br_mask])/raw_noise > sigma_threshold:
                color = 'blue'
            else:
                color = 'red'
                continue
            #TJ create normalized flux array (not generally used, but can be useful)
            Nflux = flux/continuum
            #TJ find peaks outside of 
            peaks, _ = find_peaks(Nflux, height=1+(sigma_threshold * np.std(Nflux[Br_mask])))
            # Step 4. Mask out preliminary peaks to estimate continuum noise robustly
            mask = np.ones_like(flux, dtype=bool)
            mask[peaks] = False
            noise = np.std(Nflux[Br_mask&mask])
            br_a = 0
            
            for peak in peaks:
                if (np.isclose(wl[peak], 40580e-10, 0.001)) and (flux[peak] == max(flux[peaks])):
                    left_idx = None
                    right_idx = None
                    for step in range(100):
                        continuum_L = continuum[peak-step]
                        continuum_R = continuum[peak+step]
                        flux_L = flux[peak-(step+1)]
                        flux_R = flux[peak+(step+1)]
                        if (left_idx is None) and ((flux_L < continuum[peak-(step+1)]) or ((flux[peak-(step)] > flux[peak-(step-1)]) and step>2)):
                            left_idx = peak-(step+1)
                            flux_edge = flux[left_idx:left_idx+2]
                            wl_edge = wl[left_idx:left_idx+2]
                            continuum_edge = continuum[left_idx:left_idx+2]
                            
                            f_interp = interp1d(flux_edge, wl_edge, kind='linear', bounds_error=False, fill_value="extrapolate")
                            prepend_wl = f_interp(continuum[left_idx+1])  # Or use continuum[left_idx] if you prefer
                            prepend_flux = continuum[left_idx+1]
                            if (abs(prepend_wl - wl[peak]) > 1e-8) or prepend_wl > wl[peak]:
                                prepend_wl = wl[peak-2]
                                prepend_flux = continuum[peak-2]
                                left_idx = peak-3
                                left_color = 'black'
                            else:
                                left_color = 'green'
                        if (right_idx is None) and ((flux_R < continuum[peak-(step+1)]) or ((flux[peak+(step)] > flux[peak+(step-1)]) and step>2)):
                            right_idx = peak+(step+1)
                            flux_edge = flux[right_idx-1:right_idx+1]
                            wl_edge = wl[right_idx-1:right_idx+1]
                            continuum_edge = continuum[right_idx-1:right_idx+1]
                            
                            f_interp = interp1d(flux_edge, wl_edge, kind='linear', bounds_error=False, fill_value="extrapolate")
                            append_wl = f_interp(continuum[right_idx-1])
                            append_flux = continuum[right_idx-1]
                            if (abs(append_wl - wl[peak]) > 1e-8) or (append_wl < wl[peak]):
                                append_wl = wl[peak+2]
                                append_flux = continuum[peak+2]
                                right_idx = peak+3
                                right_color = 'black'
                            else:
                                right_color = 'orange'
                        if (left_idx is not None) and (right_idx is not None):
                            break
                    bracket_fluxes = flux[left_idx+1:right_idx-1]
                    bracket_wl = wl[left_idx+1:right_idx-1]
                    bracket_fluxes = np.insert(bracket_fluxes, 0, prepend_flux)
                    bracket_wl = np.insert(bracket_wl, 0, prepend_wl)
                    bracket_fluxes = np.append(bracket_fluxes, append_flux)
                    bracket_wl = np.append(bracket_wl, append_wl)
                    bracket_continuum = continuum[left_idx+1:right_idx-1]
                    bracket_continuum = np.insert(bracket_continuum, 0, continuum[left_idx+1])
                    bracket_continuum = np.append(bracket_continuum, continuum[right_idx-1])
                    
                    continuum_subtracted = flux - continuum
                    no_bra_flux = flux.copy()
                    no_bra_flux[left_idx:right_idx] = continuum[left_idx:right_idx]
                    
                    plt.scatter(wl[peak], flux[peak], color = 'red')
                    plt.scatter(bracket_wl[0], bracket_fluxes[0], color = left_color)
                    plt.scatter(bracket_wl[-1], bracket_fluxes[-1], color = right_color)
                    plt.plot(wl, flux, color = 'black')
                    plt.plot(wl, continuum, color = 'red')
                    plt.axvline(x=wl[left_idx+1])
                    plt.axvline(x=wl[right_idx-1])
                    plt.title(f'pixel {x_pix}, {y_pix}')
                    plt.show()
                    
                    bra_only = bracket_fluxes - bracket_continuum
                    if (bra_only[0] != 0) or (bra_only[-1]!=0):
                        'Something went wrong!'
                    br_a_flux = get_Fnu_transmission(bra_only, bracket_wl, filter_trans, filter_wl)
                    total_flux = get_Fnu_transmission(flux, wl, filter_trans, filter_wl)
                    contaminate_flux = get_Fnu_transmission(no_bra_flux - continuum, wl, filter_trans, filter_wl)
                    
                    bra_flux.append(br_a_flux)
                    obs_flux.append(total_flux)
                    contamination.append(contaminate_flux)
                    print()

    print(f'Loop stopped with pixel {x_pix}, {y_pix}')
    obs_flux = np.array(obs_flux)
    contamination = np.array(contamination)
    fluxes_wo_bra = np.array(fluxes_wo_bra)
    bra_flux = np.array(bra_flux)
    np.save(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_br_a_flux_bare.npy', bra_flux)
    np.save(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_observed_bare.npy', obs_flux)
    np.save(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_contamination_bare.npy', contamination)
    np.save(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_flux_wo_br_a_bare.npy', fluxes_wo_bra)

In [ ]:

for file in files:
    prefix = file.split('jw03435-')[-1][:9]
    bra_flux = np.load(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_br_a_flux_bare.npy')
    obs_flux = np.load(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_observed_bare.npy')
    contamination = np.load(f'Data_files/misc_data/F405N_contamination/{file.split("jw03435-")[-1][:9]}_contamination_bare.npy')
    plt.scatter(obs_flux, contamination/bra_flux, marker = 'x', s = 1)
    median = np.nanmedian(contamination/bra_flux)
    upper = np.nanpercentile(contamination/bra_flux, 95)
    lower = np.nanpercentile(contamination/bra_flux, 5)
    plt.axhline(y = median)
    plt.axhline(y = upper, linestyle = '--')
    plt.axhline(y = lower, linestyle = '--')
    plt.title(f'Contamination analysis of {prefix}')
    plt.xlabel('Total Observed Flux (MJy/sr)')
    plt.ylabel('Ratio of contaminate flux to br-a flux')
    plt.ylim(-0.3, 0.3)
    plt.xscale('log')
    plt.show()
    print(lower, upper)

In [ ]:
files

In [ ]:
len(contamination)

In [ ]:
len(bra_flux)

In [ ]:
#This is a test space
show_all_plots = False
show_voigt_fit = False
show_continuum_approx = False
show_continuum_wo_br_a = False
show_normalized_flux = False

#TJ initialize output arrays that will be saved at the end of this cell
obs_flux = []
fluxes_wo_bra = []
contamination = []
discard_mask = []
bra_flux = []
#TJ cut cube so only filter's wavelengths are included
cube = SpectralCube.read(files[0], hdu = 'SCI').with_spectral_unit(u.m)
subcube = cube.spectral_slab(wl1, wl2)
#TJ set minimum continuum flux over continuum noise ratio to include pixel in analysis
sigma_threshold = 3
#TJ loop through all pixels
x_pix = 63
y_pix = 100
#TJ assign flux array for this pixel
flux = subcube[:, y_pix, x_pix]
flux = np.array(flux.value, dtype=float)
#TJ ignore pixels that include nans anywhere
wl = subcube.spectral_axis.value
Br_mask = np.ones_like(flux, dtype=bool)
#TJ create mask to ignore region directly around the Br-a feature so noise isnt super high whenever this feature is strong
Br_mask[(wl>40500e-10) & (wl<40700e-10)] = False
#TJ assign running continuum and noise in flux array (outside the Br-a feature)
continuum = median_filter(flux, size=30, mode='reflect')
raw_noise = np.std(flux[Br_mask])
if np.median(flux[Br_mask])/raw_noise > sigma_threshold:
    color = 'blue'
else:
    color = 'red'
#TJ create normalized flux array (not generally used, but can be useful)
Nflux = flux/continuum

#TJ find peaks outside of 
peaks, _ = find_peaks(Nflux, height=1+(sigma_threshold * np.std(Nflux[Br_mask])))
# Step 4. Mask out preliminary peaks to estimate continuum noise robustly
mask = np.ones_like(flux, dtype=bool)
mask[peaks] = False
noise = np.std(Nflux[Br_mask&mask])
x = np.linspace(wl[0], wl[-1], 1000)
flux_HD = np.interp(x, wl, flux)
test_Br_mask = np.ones_like(flux_HD, dtype=bool)
test_Br_mask[(x>40500e-10) & (x<40700e-10)] = False

test_peaks, _ = find_peaks(flux_HD, height=1+(sigma_threshold * np.std(flux_HD[test_Br_mask])))

br_a = 0

for peak in peaks:
    if np.isclose(wl[peak], 40580e-10, 0.001):
        br_peak = peak
        br_a = fit_voigt_to(wl[peak], 1, wl, flux, show_plot = (show_all_plots or show_voigt_fit), feature_idx_width=10)
        br_a_x, br_a_y = br_a[0]
        line = voigt(x, *br_a[4])

        flux_wo_br_a = flux_HD - line
        br_a_continuum = np.interp(br_a_x, wl, continuum)
        continuum_HD = np.interp(x, wl, continuum)
        continuum_wo_br_a = median_filter(flux_wo_br_a, size=300, mode='reflect')
        residuals = flux_HD - continuum_HD - line
        if np.median(flux[Br_mask])/raw_noise > sigma_threshold:
            discard_mask.append(True)
        else:
            discard_mask.append(False)
        bra_flux.append(br_a[1])
        obs_flux.append(get_Fnu_transmission(flux_HD, x, filter_trans, filter_wl))
        fluxes_wo_bra.append(get_Fnu_transmission(flux_wo_br_a, x, filter_trans, filter_wl))
        contamination.append(get_Fnu_transmission((flux_wo_br_a - continuum_wo_br_a), x, filter_trans, filter_wl))
        print()
for test_peak in test_peaks:
    if np.isclose(x[test_peak], 40580e-10, rtol=1e-03, atol=1e-9):
        br_a = fit_voigt_to(wl[br_peak], 1, wl, flux, show_plot = (show_all_plots or show_voigt_fit), feature_idx_width=10)
        test_br_a = fit_voigt_to(x[test_peak], 1, x, flux_HD, show_plot = (show_all_plots or show_voigt_fit), feature_idx_width=75)
        line = voigt(x, *br_a[4])
        test_line = voigt(x, *test_br_a[4])
        plt.plot(x, test_line+continuum_wo_br_a, color = 'red')
        plt.plot(x, line+continuum_wo_br_a, color = 'blue')
        plt.plot(x, flux_HD, color = 'black')
        plt.show()
